# Code used in Meisler, Gabrieli, and Christodoulou 202X

In [ ]:
# Import packages
import os
import os.path as op
import numpy as np
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt
from matplotlib import rcParams
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import scipy
import scipy.stats as st
from glob import glob
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

# Define tracts of interest with TRACULA naming convention
tracts = {'Left_AF':'lh.af',
          'Right_AF':'rh.af',
          'Left_ILF':'lh.ilf',
          'Right_ILF':'rh.ilf',
          'Left_CST':'lh.cst',
          'Right_CST':'rh.cst',
          'Splenium':'cc.splenium'}

## Load in Dataframe if already made to save time

In [ ]:
df = pd.read_csv('df_share.csv') # This is the DF from the repo
df = df.set_index('subjects')
df['All']=['All' for i in range(len(df))]

## Rescale data (z-score continuous columns)

In [ ]:
# Don't try to rescale binary columns
keys_to_not_standardize = ['Sex','Handedness','Intervention','ADHD','All']
keys_to_standardize = [key for key in df.keys() if key not in keys_to_not_standardize]
df_std = df[keys_to_not_standardize]
for key in keys_to_standardize:
    df_std[key] = preprocessing.scale(df[key], with_mean=True, with_std=True)
    
# Get separate dataframes for intervention and no intervention
df_int = df_std[df_std['Intervention']=='Y']
df_no_int = df_std[df_std['Intervention']=='N']

## Print out group summary stats

In [ ]:
# t-tests
for key in ['Age_pre','SES', 'SWE2_ss_pre', 'SWE2_ss_post', 'PDE2_ss_pre', 'PDE2_ss_post', 
            'W3WI_ss_pre', 'W3WI_ss_post', 'W3WA_ss_pre', 'W3WA_ss_post', 'SIT_ss_pre', 'SIT_ss_post', 'SIT_ss_diff',
           'Composite_Score_ss_pre', 'Composite_Score_ss_post', 'Composite_Score_ss_diff', ]:
    print(key)
    stat_grouped = df.groupby('Intervention')[key].agg(['mean', 'sem'])
    stat_all = df.groupby('All')[key].agg(['mean', 'sem'])
    stat_combined = pd.concat([stat_all,stat_grouped])
    x = df[key][df['Intervention']=='Y']
    y = df[key][df['Intervention']=='N']
    group_comparison = pg.ttest(x, y)

    print(stat_combined,'\n',group_comparison,'\n')

## Run Models

#### OLS for correlations

In [ ]:
for tract in tracts.keys():
    # CHANGE THE FOLLOWING 5 VARIABLES AS NEEDED
    data = df_std # df_std for all subjects with z-scoring, df for all subjects without standardization, 
                  # df_int for just intervention, df_no_int for just no intervention
    session = 'diff' # 'pre', 'post', or 'diff'
    metric = 'MD' # 'MD' or 'FA'
    test = 'SIT' #'Composite_Score', 'SIT', or other reading scores of interest
    scoretype = 'ss' # 'ss' for standard score, 'raw' for raw score
    
    formula_base = f"{tract}_{metric}_{session} ~ Sex"
    # If looking at difference, add age at first scan and both TMI time points
    if session == 'diff':
        formula_reduced = formula_base + ' + Age_pre + TMI_pre + TMI_post'
    # Otherwise, for cross-sectional data add Age and TMI for that given time point
    else:
        formula_reduced = formula_base + f' + Age_{session} + TMI_{session}'
    formula_full = formula_reduced + f" + {test}_{scoretype}_{session}"

    # Run the full model
    model_full = smf.ols(formula=formula_full, data=data).fit()
    r2_full = model_full.rsquared_adj
    # Run the reduced model
    model_reduced = smf.ols(formula=formula_reduced, data=data).fit()
    r2_reduced = model_reduced.rsquared_adj
    # Get effect size
    r2_diff = r2_full - r2_reduced
    print('\n'+tract, 'FORMULA:',formula_full)
    print('\n R2-adj for Reading Term:',r2_diff,'\n')
    print(model_full.summary())
    
    ## Uncomment if you want to create plots with each model
    #fig = plt.figure(figsize=(8, 6))
    #rcParams.update({'font.size': 10})
    #sm.graphics.plot_regress_exog(model, f'{test}_{scoretype}_{session}', fig=fig)
    #plt.show()

## Regression Plots

In [ ]:
fig_frame = plt.figure(figsize=(10, 8))
ax=fig_frame.add_axes([0,0,1,1])
data = df # df_std for all subjects with z-scoring, df for all subjects without standardization, 
                  # df_int for just intervention, df_no_int for just no intervention
hemi = 'Left' # 'Left' or 'Right'
tract = 'Splenium' # 'Left_AF', 'Right_AF', 'Left_ILF', 'Right_ILF', 'Left_CST', 'Right_CST', 'Splenium'
metric = 'MD' # 'MD' or 'FA'
session = 'diff' # 'pre', 'post', or 'diff'
test = 'Composite_Score' # 'Composite_Score', 'SIT', or other reading scores of interest
scoretype = 'ss' # 'ss' for standard score, 'raw' for raw score
int_color = 'purple' # color for intervention group
no_int_color = 'y' # color for no intervention group

int_line = True
rcParams.update({'font.size': 20})

if session == 'diff':
    covars = ['Sex', 'Age_pre', 'TMI_pre', 'TMI_post']
else:
    covars = ['Sex', f'Age_{session}', f'TMI_{session}']

labels = df['Intervention']
intervention_inds = (labels=='Y')

colors = [int_color if group=='Y' else no_int_color for group in labels]

if tract != 'Splenium':
    endog_label = f'{hemi}_{tract}_{metric}_{session}'
else:
    endog_label = f'{tract}_{metric}_{session}' # no hemi label for splenium
    
    
fig, coords = statsmodels.graphics.regressionplots.plot_partregress(endog=f'{endog_label}', exog_i=f'{test}_{scoretype}_{session}',
                                                      exog_others = covars, data=data, ax=ax, obs_labels=False, ret_coords=True, marker=None)
x_coords = coords[0]
y_coords = coords[1]

#plt.scatter(x_coords, y_coords, c = colors)
#plt.show()
plt.cla()
if metric == 'MD':
    ax.ticklabel_format(axis='y', style='', scilimits=(0,0), useOffset=None, useLocale=None, useMathText=True)
sns.regplot(data=None, x=x_coords, y=y_coords, truncate = True, line_kws={'color':'k','linestyle':'-'}, scatter_kws={'color':colors})
#plt.legend(['Intervention','No Intervention'])
sns.regplot(data=None, x=x_coords[intervention_inds], y=y_coords[intervention_inds], truncate = True, scatter = False, line_kws={'color':int_color,'linestyle':'--'})

if scoretype == 'ss':
    test_label = 'Standardized'
elif scoretype =='raw' :
    test_label += 'Raw'
if test == 'SIT':
    test_label += ' Symbol Imagery Test'
elif test == 'Composite_Score':
    test_label += ' Composite Reading Index'


if session == 'diff':
    xlabel = f'Δ {test_label} Scores (Residuals)'
    if tract != 'Splenium':
        ylabel = f'Δ {metric} in {hemi} {tract} (Residuals)'
    else:
        ylabel = f'Δ {metric} in {tract} (Residuals)'
else:
    xlabel = f'{test_label} Scores at Time {session.capitalize()} (Residuals)'
    if tract != 'Splenium':
        ylabel = f'{metric} in {hemi} {tract} at Time {session.capitalize()} (Residuals)'
    else:
        ylabel = f'{metric} in {tract} at Time {session.capitalize()} (Residuals)'
        

plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.savefig(f'figures/{test}_{scoretype}_{session}_{metric}_{hemi}_{tract}.svg')
plt.show()

### Plot changes in SIT scores

In [ ]:
# Create figure/axes object
fig, ax = plt.subplots(figsize= (14, 10))

# Colors for plot
BG_WHITE = "#fbf9f4"
GREY_DARK = "#747473"
# Set background color
fig.patch.set_facecolor('white')
ax.set_facecolor('white')

# Colors for participant data
COLOR_SCALE = ["purple", "purple", "y", "y"]

tick_len = 0.5 # For signifiance markers

# Horizontal positions for the violins. 
POSITIONS = [0, 1, 2, 3]

# Get reading data to plot for intervention and no intervention groups
times = ['pre', 'post']
score = "SIT_ss"
scores_int = [df[df["Intervention"] == "Y"][f"{score}_{time}"].values for time in times]
scores_noint = [df[df["Intervention"] == "N"][f"{score}_{time}"].values for time in times]

# Define x position of dots with a jitter
jitter = 0.03
x_data_int = [np.array([i] * len(d)) for i, d in enumerate(scores_int)]
x_data_noint = [np.array([i+2] * len(d)) for i, d in enumerate(scores_noint)]
x_jittered_int = [x + st.t(df=6, scale=jitter).rvs(len(x)) for x in x_data_int]
x_jittered_noint = [x + st.t(df=6, scale=jitter).rvs(len(x)) for x in x_data_noint]

### Add intervention data

# Add violins
violins = ax.violinplot(
    scores_int, 
    positions=POSITIONS[0:2],
    widths=0.45,
    bw_method="silverman",
    showmeans=False, 
    showmedians=False,
    showextrema=False
)
# Customize violins (remove fill, customize line, etc.)
for pc in violins["bodies"]:
    pc.set_facecolor("none")
    pc.set_edgecolor("BLACK")
    pc.set_linewidth(1.4)
    pc.set_alpha(1)
    
# Add boxplots
medianprops = dict(
    linewidth=4, 
    color=GREY_DARK,
    solid_capstyle="butt"
)
boxprops = dict(
    linewidth=2, 
    color=GREY_DARK
)

ax.boxplot(
    scores_int,
    positions=POSITIONS[0:2], 
    showfliers = False, # Do not show the outliers beyond the caps.
    showcaps = False,   # Do not show the caps
    medianprops = medianprops,
    whiskerprops = boxprops,
    boxprops = boxprops
)

# Add jittered dots
for x, y, color in zip(x_jittered_int, scores_int, COLOR_SCALE[0:2]):
    ax.scatter(x, y, s = 100, color=color, alpha=0.6, edgecolors='k')
# Add lines connecting the pre-post pairs
for i in range(np.shape(x_jittered_int)[1]):
    plt.plot([x_jittered_int[0][i], x_jittered_int[1][i]], [scores_int[0][i], scores_int[1][i]], color='gray', alpha=0.5)
    
### Add no intervention data
    
# Add violins
violins = ax.violinplot(
    scores_noint, 
    positions=POSITIONS[2:],
    widths=0.45,
    bw_method="silverman",
    showmeans=False, 
    showmedians=False,
    showextrema=False
)
# Customize violins (remove fill, customize line, etc.)
for pc in violins["bodies"]:
    pc.set_facecolor("none")
    pc.set_edgecolor("BLACK")
    pc.set_linewidth(1.4)
    pc.set_alpha(1)
    
# Add boxplots
medianprops = dict(
    linewidth=4, 
    color=GREY_DARK,
    solid_capstyle="butt"
)
boxprops = dict(
    linewidth=2, 
    color=GREY_DARK
)

ax.boxplot(
    scores_noint,
    positions=POSITIONS[2:], 
    showfliers = False, # Do not show the outliers beyond the caps.
    showcaps = False,   # Do not show the caps
    medianprops = medianprops,
    whiskerprops = boxprops,
    boxprops = boxprops
)

# Add jittered dots
for x, y, color in zip(x_jittered_noint, scores_noint, COLOR_SCALE[2:]):
    ax.scatter(x, y, s = 100, color=color, alpha=0.6, edgecolors='k')
# Add lines connecting the pre-post pairs
for i in range(np.shape(x_jittered_noint)[1]):
    plt.plot([x_jittered_noint[0][i], x_jittered_noint[1][i]], [scores_noint[0][i], scores_noint[1][i]], color='gray', alpha=0.5)
    
### Add some labels and text to the plot
plt.ylabel('SIT Score (Standardized)')
plt.xticks(ticks=POSITIONS, labels=['Pre', 'Post', 'Pre', 'Post'])
txt="Intervention"
plt.figtext(.32, 0.01, txt, wrap=True, horizontalalignment='center', fontsize=20)
txt="No Intervention"
plt.figtext(.7, 0.01, txt, wrap=True, horizontalalignment='center', fontsize=20)

### Add some significance brackets
# For paired t-test
height = 125
ax.plot([0, 0, 1, 1], [height - tick_len, height, height, height - tick_len], c="black")
test = st.ttest_rel(scores_int[0], scores_int[1])
txt=f"Paired t-test, t={test.statistic:.3f}, p={test.pvalue:.3f}"
plt.figtext(.32, .8, txt, wrap=True, horizontalalignment='center', fontsize=12)
# For two-sample t-test
height = 130
ax.plot([1, 1, 3, 3], [height - tick_len, height, height, height - tick_len], c="black")
test = st.ttest_ind(scores_int[1], scores_noint[1])
txt=f"Two-sample t-test, t={test.statistic:.3f}, p={test.pvalue:.3f}"
plt.figtext(.6, .855, txt, wrap=True, horizontalalignment='center', fontsize=12)

plt.savefig(f'figures/SIT_score_figure.png', bbox_inches='tight')
plt.show()

### Plot changes in Composite reading scores

In [ ]:
# Create figure/axes object
fig, ax = plt.subplots(figsize= (14, 10))

# Colors for plot
BG_WHITE = "#fbf9f4"
GREY_DARK = "#747473"
# Set background color
fig.patch.set_facecolor('white')
ax.set_facecolor('white')

# Colors for participant data
COLOR_SCALE = ["purple", "purple", "y", "y"]

tick_len = 0.5 # For signifiance markers

# Horizontal positions for the violins. 
POSITIONS = [0, 1, 2, 3]

# Get reading data to plot for intervention and no intervention groups
times = ['pre', 'post']
score = "Composite_Score_ss"
scores_int = [df[df["Intervention"] == "Y"][f"{score}_{time}"].values for time in times]
scores_noint = [df[df["Intervention"] == "N"][f"{score}_{time}"].values for time in times]

# Define x position of dots with a jitter
jitter = 0.03
x_data_int = [np.array([i] * len(d)) for i, d in enumerate(scores_int)]
x_data_noint = [np.array([i+2] * len(d)) for i, d in enumerate(scores_noint)]
x_jittered_int = [x + st.t(df=6, scale=jitter).rvs(len(x)) for x in x_data_int]
x_jittered_noint = [x + st.t(df=6, scale=jitter).rvs(len(x)) for x in x_data_noint]

### Add intervention data

# Add violins
violins = ax.violinplot(
    scores_int, 
    positions=POSITIONS[0:2],
    widths=0.45,
    bw_method="silverman",
    showmeans=False, 
    showmedians=False,
    showextrema=False
)
# Customize violins (remove fill, customize line, etc.)
for pc in violins["bodies"]:
    pc.set_facecolor("none")
    pc.set_edgecolor("BLACK")
    pc.set_linewidth(1.4)
    pc.set_alpha(1)
    
# Add boxplots
medianprops = dict(
    linewidth=4, 
    color=GREY_DARK,
    solid_capstyle="butt"
)
boxprops = dict(
    linewidth=2, 
    color=GREY_DARK
)

ax.boxplot(
    scores_int,
    positions=POSITIONS[0:2], 
    showfliers = False, # Do not show the outliers beyond the caps.
    showcaps = False,   # Do not show the caps
    medianprops = medianprops,
    whiskerprops = boxprops,
    boxprops = boxprops
)

# Add jittered dots
for x, y, color in zip(x_jittered_int, scores_int, COLOR_SCALE[0:2]):
    ax.scatter(x, y, s = 100, color=color, alpha=0.6, edgecolors='k')
# Add lines connecting the pre-post pairs
for i in range(np.shape(x_jittered_int)[1]):
    plt.plot([x_jittered_int[0][i], x_jittered_int[1][i]], [scores_int[0][i], scores_int[1][i]], color='gray', alpha=0.5)
    
### Add no intervention data
    
# Add violins
violins = ax.violinplot(
    scores_noint, 
    positions=POSITIONS[2:],
    widths=0.45,
    bw_method="silverman",
    showmeans=False, 
    showmedians=False,
    showextrema=False
)
# Customize violins (remove fill, customize line, etc.)
for pc in violins["bodies"]:
    pc.set_facecolor("none")
    pc.set_edgecolor("BLACK")
    pc.set_linewidth(1.4)
    pc.set_alpha(1)
    
# Add boxplots
medianprops = dict(
    linewidth=4, 
    color=GREY_DARK,
    solid_capstyle="butt"
)
boxprops = dict(
    linewidth=2, 
    color=GREY_DARK
)

ax.boxplot(
    scores_noint,
    positions=POSITIONS[2:], 
    showfliers = False, # Do not show the outliers beyond the caps.
    showcaps = False,   # Do not show the caps
    medianprops = medianprops,
    whiskerprops = boxprops,
    boxprops = boxprops
)

# Add jittered dots
for x, y, color in zip(x_jittered_noint, scores_noint, COLOR_SCALE[2:]):
    ax.scatter(x, y, s = 100, color=color, alpha=0.6, edgecolors='k')
# Add lines connecting the pre-post pairs
for i in range(np.shape(x_jittered_noint)[1]):
    plt.plot([x_jittered_noint[0][i], x_jittered_noint[1][i]], [scores_noint[0][i], scores_noint[1][i]], color='gray', alpha=0.5)
    
### Add some labels and text to the plot
plt.ylabel('Composite Reading Index (Standardized)')
plt.xticks(ticks=POSITIONS, labels=['Pre', 'Post', 'Pre', 'Post'])
txt="Intervention"
plt.figtext(.32, 0.01, txt, wrap=True, horizontalalignment='center', fontsize=20)
txt="No Intervention"
plt.figtext(.7, 0.01, txt, wrap=True, horizontalalignment='center', fontsize=20)

### Add some significance brackets
# For paired t-test
height = 100
ax.plot([2, 2, 3, 3], [height - tick_len, height, height, height - tick_len], c="black")
test = st.ttest_rel(scores_noint[0], scores_noint[1])
txt=f"Paired t-test, t={test.statistic:.3f}, p={test.pvalue:.5f}"
plt.figtext(.705, .855, txt, wrap=True, horizontalalignment='center', fontsize=12)

plt.savefig(f'figures/Composite_score_figure.png', bbox_inches='tight')
plt.show()